<a href="https://colab.research.google.com/github/sjung-stat/Customer-Support-Chat-Intent-Classification/blob/main/Data%20Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd


### Import dataset from GitHub 

In [ ]:
url_training = 'https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv'
url_testing = 'https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv'
df_training = pd.read_csv(url_training)
df_testing = pd.read_csv(url_testing)

In [ ]:
intent_examples_count = df_training.groupby(['category'])['category'].count().reset_index(name='count').sort_values(['count'], ascending=False)

In [ ]:
with pd.option_context('display.max_rows', None):
  print(intent_examples_count)

                                            category  count
16                          card_payment_fee_charged    187
29               direct_debit_payment_not_recognised    182
7   balance_not_updated_after_cheque_or_cash_deposit    181
75                     wrong_amount_of_cash_received    180
20                            cash_withdrawal_charge    177
63                         transaction_charged_twice    175
27                          declined_cash_withdrawal    173
64                              transfer_fee_charged    172
66                transfer_not_received_by_recipient    171
6            balance_not_updated_after_bank_transfer    171
52                                    request_refund    169
17                       card_payment_not_recognised    168
18                  card_payment_wrong_exchange_rate    167
35                         extra_charge_on_statement    166
76           wrong_exchange_rate_for_cash_withdrawal    163
0                              Refund_no

In [ ]:
# card_arrival vs. card_delivery_estimate ==> look similar ==> compare the text data of each category

df_training[df_training['category']=='card_arrival']

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival
...,...,...
148,I haven't gotten my credit card in the mail.,card_arrival
149,My card hasn't arrived yet. What do I do?,card_arrival
150,"My card still hasn't arrived after two weeks, ...",card_arrival
151,My card appears to have never arrived?,card_arrival


In [ ]:
df_training[df_training['category']=='card_delivery_estimate']

,text,category
1006,Can it specifically be delivered on a certain ...,card_delivery_estimate
1007,I need to get my card quickly,card_delivery_estimate
1008,When can I expect my card? I live in the US.,card_delivery_estimate
1009,Help! When will the card arrive at my home?,card_delivery_estimate
1010,Can I get my card fast tracked?,card_delivery_estimate
...,...,...
1113,what is the leadtime for a new card to show up...,card_delivery_estimate
1114,What's the earliest time to receive my card?,card_delivery_estimate
1115,are you sending me my card?,card_delivery_estimate
1116,I need my card immediately.,card_delivery_estimate


In [ ]:
df_training.isnull().sum()

text        0
category    0
dtype: int64

In [ ]:
!pip install transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 32.2 MB/s 
     |████████████████████████████████| 182 kB 28.1 MB/s 


In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Artificial Intelligence [MASK] take over the world.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.31824034452438354,
  'token': 2064,
  'token_str': 'can',
  'sequence': 'artificial intelligence can take over the world.'},
 {'score': 0.1829964816570282,
  'token': 2097,
  'token_str': 'will',
  'sequence': 'artificial intelligence will take over the world.'},
 {'score': 0.05600164458155632,
  'token': 2000,
  'token_str': 'to',
  'sequence': 'artificial intelligence to take over the world.'},
 {'score': 0.04519513249397278,
  'token': 2015,
  'token_str': '##s',
  'sequence': 'artificial intelligences take over the world.'},
 {'score': 0.04515312612056732,
  'token': 2052,
  'token_str': 'would',
  'sequence': 'artificial intelligence would take over the world.'}]

In [ ]:
# Required to preprocess text for BERT inputs
!pip install -q tensorflow-text==2.6.0

     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 570.3 MB 328 bytes/s 
     |████████████████████████████████| 462 kB 42.5 MB/s 
     |████████████████████████████████| 14.8 MB 471 kB/s 
     |████████████████████████████████| 5.6 MB 28.6 MB/s 
     |████████████████████████████████| 132 kB 34.6 MB/s 
     |████████████████████████████████| 1.3 MB 41.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
pydantic 1.10.2 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
jaxlib 0.3.22+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.3.23 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.0.8 requires n

## Tast 1: 
See if some of the categories can be merged together to have enough data for each category.

## Task 2:
Consider dropping some of the minor categories to reduce complexity of the data

## Task 3:
Do some research on "zero shot intent classification"


Notes:

- Frameworks:
  - Address the difference between ***TensorFlow*** and ***PyTorch*** frameworks and justify the reason why I would choose one over the other
    - I would use TensorFlow because of its 모델 배포 (TensorFlow Serving)
- Transformers:
  - GPT-3, BERT, etc.
  - which one should I use for intent recognition?
- How to incorporate zero-shot approach
:




Example repositories:
- Intent Classification with BERT
  - [https://hannibunny.github.io/mlbook/transformer/intent_classification_with_bert.html]
